In [5]:
import rasterio
from rasterio.features import geometry_mask
import numpy as np
import geopandas as gpd

# Load raster and shapefile
raster_path = "data/raster_grid.tif"
shapefile_path = "data/bwi.gpkg"

# Open raster dataset
with rasterio.open(raster_path) as src:
    # Read raster data
    raster_data = src.read(1)  # Assuming a single-band raster

    # Retrieve raster metadata
    profile = src.profile

# Load GeoPackage into a GeoDataFrame
gdf = gpd.read_file(shapefile_path)

# Calculate raster extent
xmin, ymin = profile['transform'] * (0, 0)
xmax, ymax = profile['transform'] * (src.width, src.height)

# Create an empty array to store mean values
mean_values = np.zeros_like(raster_data, dtype=np.float32)

# Iterate over each polygon and calculate mean value within its extent
for idx, geom in gdf.iterrows():  # Iterate over rows of GeoDataFrame directly
    # Create a mask for the current polygon
    mask = geometry_mask([geom['geometry']], (src.height, src.width), transform=profile['transform'], invert=False)
    
    # Extract values within the polygon's mask
    values_within_polygon = raster_data[mask]
    
    # Extract attribute values from GeoDataFrame
    attribute_values = geom['vPicea']  # Assuming "vPicea" is the column name
    
    # Calculate mean value
    mean_value = np.mean(attribute_values)
    
    # Assign mean value to the corresponding cells in the empty array
    mean_values[mask] = mean_value

# Write the array to a new raster file
output_raster_path = "data/output_raster.tif"  # Define the output raster path
with rasterio.open(output_raster_path, 'w', **profile) as dst:
    dst.write(mean_values, 1)


In [1]:
import rasterio
from rasterio.features import geometry_mask
import numpy as np
import geopandas as gpd

# Load raster and shapefile
raster_path = "data/raster_grid.tif"
shapefile_path = "data/bwi.gpkg"

# Open raster dataset
with rasterio.open(raster_path) as src:
    # Read raster data
    raster_data = src.read(1)  # Assuming a single-band raster
    profile = src.profile

# Load GeoDataFrame
gdf = gpd.read_file(shapefile_path)

# Get raster window size
win_xsize, win_ysize = profile["width"], profile["height"]

# Create an empty array to store mean values
mean_values = np.zeros_like(raster_data, dtype=np.float32)

# Iterate over each polygon and compute mean value for intersecting cells
for idx, geom in gdf.iterrows():
    # Extract attribute values
    attribute_values = geom["vPicea"]
    
    # Create mask for the current polygon
    mask = geometry_mask([geom["geometry"]], transform=profile["transform"], invert=False, out_shape=raster_data.shape)
    
    # Multiply the mask with the attribute values to get values within the polygon
    values_within_polygon = raster_data * mask
    
    # Update mean_values array with the values within the polygon
    mean_values += values_within_polygon
    
# Calculate the number of cells within each polygon
num_cells_within_polygon = geometry_mask(gdf["geometry"], transform=profile["transform"], invert=False, out_shape=raster_data.shape)

# Compute mean value for each cell by dividing the sum of values within each polygon by the number of cells within the polygon
mean_values /= num_cells_within_polygon

# Write the array to a new raster file
output_raster_path = "data/output_raster.tif"  # Define the output raster path
with rasterio.open(output_raster_path, 'w', **profile) as dst:
    dst.write(mean_values, 1)
